In [1]:
import pandas as pd
import yfinance as yf

# Define the ticker symbol and time period
ticker = "AAPL"
start_date = "2010-01-01"
end_date = "2023-05-20"

# Fetch the historical stock data using Yahoo Finance API
stock_data = yf.download(ticker, start=start_date, end=end_date)

# Calculate Keltner Channel
def calculate_keltner_channel(data, n, k):
    data['Typical Price'] = (data['High'] + data['Low'] + data['Close']) / 3
    data['MA'] = data['Typical Price'].rolling(window=n).mean()
    data['ATR'] = data['High'] - data['Low']
    data['Upper KC'] = data['MA'] + k * data['ATR']
    data['Lower KC'] = data['MA'] - k * data['ATR']
    return data

# Calculate Bollinger Bands
def calculate_bollinger_bands(data, n, std_dev):
    data['MA'] = data['Close'].rolling(window=n).mean()
    data['Std'] = data['Close'].rolling(window=n).std()
    data['Upper BB'] = data['MA'] + std_dev * data['Std']
    data['Lower BB'] = data['MA'] - std_dev * data['Std']
    return data

# Calculate MACD
def calculate_macd(data, short_ma, long_ma, signal_ma):
    data['EMA_short'] = data['Close'].ewm(span=short_ma, adjust=False).mean()
    data['EMA_long'] = data['Close'].ewm(span=long_ma, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal Line'] = data['MACD'].ewm(span=signal_ma, adjust=False).mean()
    return data

# Define the objective function for Bollinger Bands optimization
def calculate_bollinger_bands_returns(data, upper_threshold, lower_threshold):
    data['Position'] = 0
    data.loc[data['Close'] > data['Upper BB'], 'Position'] = -1
    data.loc[data['Close'] < data['Lower BB'], 'Position'] = 1
    data['Returns'] = data['Position'].shift(1) * data['Close'].pct_change()
    data['Cumulative Returns'] = (1 + data['Returns']).cumprod()
    return data['Cumulative Returns'].iloc[-1]

# Set up the parameter space for Bollinger Bands optimization
n_values = [10, 20, 30]  # Periods for moving average
std_dev_values = [1, 2, 3]  # Standard deviation factors

# Calculate Keltner Channel
n_kc = 20  # Period for moving average
k_kc = 2.0  # Keltner Channel factor
stock_data = calculate_keltner_channel(stock_data, n_kc, k_kc)

# Calculate Bollinger Bands
n_bb = 20  # Period for moving average
std_dev_bb = 2.0  # Standard deviation factor
stock_data = calculate_bollinger_bands(stock_data, n_bb, std_dev_bb)

# Calculate MACD
short_ma_macd = 12  # Short-term moving average period
long_ma_macd = 26  # Long-term moving average period
signal_ma_macd = 9  # Signal line moving average period
stock_data = calculate_macd(stock_data, short_ma_macd, long_ma_macd, signal_ma_macd)

# Perform parameter optimization for Bollinger Bands
best_returns = -float('inf')
best_n = None
best_std_dev = None

for n in n_values:
    for std_dev in std_dev_values:
        # Calculate Bollinger Bands with current parameters
        bollinger_bands_data = calculate_bollinger_bands(stock_data.copy(), n, std_dev)

        # Calculate strategy returns with current parameters
        returns = calculate_bollinger_bands_returns(bollinger_bands_data.copy(), -1, 1)

        # Check if current parameters yield better returns
        if returns > best_returns:
            best_returns = returns
            best_n = n
            best_std_dev = std_dev

# Print the optimized parameters and returns for Bollinger Bands
print(f"Optimized Parameters for Bollinger Bands: n = {best_n}, std_dev = {best_std_dev}")
print(f"Strategy Returns for Bollinger Bands: {best_returns}")


[*********************100%***********************]  1 of 1 completed
Optimized Parameters for Bollinger Bands: n = 30, std_dev = 2
Strategy Returns for Bollinger Bands: 1.3077013773573762
